In [3]:
#tools and test methods
import numpy as np
import scipy.io.wavfile as wav
import random
from audiomentations import AddBackgroundNoise

C:\ProgramData\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


#### Test audiomentation

In [14]:
transform = AddBackgroundNoise(
    sounds_path="audiomentation/addedNoise8kHz",
    min_snr_in_db=-44.7,
    max_snr_in_db=-40.7,
    p=1.0
)

toAugmentPath = "audiomentation/sirene_8kHz_SNR.wav"

fr, signal = wav.read(toAugmentPath)

augmented_sound = transform(signal[0:16000], sample_rate=8000)

In [15]:
wav.write("audiomentation/sireneAugmented_8kHz_SNR.wav", 8000, augmented_sound.astype(np.int16))

#### Séparation d'un jeu de données en jeux de données Train et Test suivant un pourcentage donné

In [4]:
#séparation classe Sirènes (changer les path pour séparer une autre classe)
percentage = 0.10

f = open("datasetTrainTestToAugment/sirene_8kHz_40gain_rp_trimed_audacity.rw","r")
# f = open("datasetTrainTestToAugment/autre_8kHz_40gain_rp_complet.rw","r")
audioSize = 8000
lines = f.readlines()
linesLength = int(len(lines)/audioSize)
filePrct = int(percentage*linesLength)
testSamples = random.sample(range(0, linesLength-1), filePrct)
testSamples.sort(reverse=True)
dataTest = []
for i in testSamples :
    for j in range(i*audioSize,i*audioSize+audioSize):
        dataTest.append(lines[j])
    del lines[i*audioSize:i*audioSize+audioSize]
f.close()

In [6]:
with open("datasetTrainTestToAugment/sirene_8kHz_Test.rw", "w") as file:
# with open("datasetTrainTestToAugment/autre_8kHz_Test.rw", "w") as file:
    for data in dataTest :
        file.write(data)
with open("datasetTrainTestToAugment/sirene_8kHz_Train.rw", "w") as file:
# with open("datasetTrainTestToAugment/autre_8kHz_Train.rw", "w") as file:
    for data in lines :
        file.write(data)

#### .rw vers .wav (pour utiliser Audiomentation)

In [7]:
# Signal from the Nano RP2040 Connect being in 16 bit PCM format, conversion directly possible in .wav
name = "datasetTrainTestToAugment/sirene_8kHz_Test"
f = open(name+".rw","r")
lines = f.readlines()
# print(len(lines))
signal = []
for i in range (len(lines)) :
    s = lines[i].replace("\n","").replace("\r","")
    signal.append(int(s))

In [8]:
signalNp = np.asarray(signal)
print(len(signalNp)/8000)

wav.write(name+".wav", 8000, signalNp.astype(np.int16))
f.close()

56.0


##### print des données PCM et du .wav pour vérifier que les données ne sont pas affectées par la conversion

In [12]:
sr, signal = wav.read(name+".wav")
print(signal[0:100])
print(lines[0:100])

[-5120 -2731    12   686   169 -1243 -2726 -3443 -3722 -2433 -1053 -3750
 -5110 -1160  1293  1383    98 -1442   181  1236   738  1857  4164  2685
 -3110 -5287 -1661  1217   635 -1798 -1924   244  1707   292   188  1541
 -3613 -8333 -5079   397  2074 -1038 -3801 -1976  -731  -199  3262  4559
   563 -2793 -1347  3168  4351  2625  1976  1106    48  -526  -981   225
  -367 -4817 -5244  -266  3473  2245  -786 -1055  1038   803 -1793 -4482
 -6721 -7418 -6189 -1133  4302  4186  2262  1511   347 -1788 -2048 -1770
 -3934 -6524 -4836  2178  8361  8805  8679  7477  2836 -1618 -2153   931
  1254 -3399 -5991 -3019]
['-5120\n', '-2731\n', '12\n', '686\n', '169\n', '-1243\n', '-2726\n', '-3443\n', '-3722\n', '-2433\n', '-1053\n', '-3750\n', '-5110\n', '-1160\n', '1293\n', '1383\n', '98\n', '-1442\n', '181\n', '1236\n', '738\n', '1857\n', '4164\n', '2685\n', '-3110\n', '-5287\n', '-1661\n', '1217\n', '635\n', '-1798\n', '-1924\n', '244\n', '1707\n', '292\n', '188\n', '1541\n', '-3613\n', '-8333\n', '-

#### Augmentation d'un jeu de test et enregistrement (pour avoir des augmentations non variables)

##### Utilisable avec le dossier "background noises" et présent dans le notebook ClassificationBinaire (méthode accessible et directement utilisable depuis ce notebook)

In [ ]:
def audio_augmentation (audio, sr = 8000, proba = 0.5):
    transform = AA.AddBackgroundNoise(
        sounds_path="datasetTrainTestToAugment/backgroundNoise",
        min_snr_db=-44.7,
        max_snr_db=-40.7,
        p=proba
        )

    augmented_sound = transform(audio, sample_rate=sr)

    return augmented_sound

In [ ]:
def cut_and_augment_test_audio (audio, probability):
    audios = []
    
    for n in range(int(len(audio)/nbr_values)):
        sample = audio[int(n*nbr_values):int((n)*nbr_values+nbr_values)]

        #augmentation de l'audio
        sample = audio_augmentation(np.asarray(sample).astype(np.int16), proba = probability)
        
        if(not np.isnan(sample[0])):
            audios.append(sample)
            
    return np.asarray(audios).flatten()

In [ ]:
#enregistrements de features de jeu de test avec plusieurs degrés d'augmentation (0%, 50% et 100%)
#opération à ne faire qu'une fois par jeu de test

siren_path_test = "datasetTrainTestToAugment/sirene_8kHz_Test.rw"
other_path_test = "datasetTrainTestToAugment/autre_8kHz_Test.rw"

signalTEMP = get_signal(siren_path_test)

#cut_and_augment_test_audio : méthode à instancier plus bas

#augmentations et sauvegarde du jeu de test augmenté
wav.write("datasetTrainTestToAugment/sirene_8kHz_Test_Augmented_p1.wav", 8000, cut_and_augment_test_audio(signalTEMP,1).astype(np.int16))
wav.write("datasetTrainTestToAugment/sirene_8kHz_Test_Augmented_p05.wav", 8000, cut_and_augment_test_audio(signalTEMP,0.5).astype(np.int16))

signalTEMP = get_signal(other_path_test)

#augmentations et sauvegarde du jeu de test augmenté
wav.write("datasetTrainTestToAugment/autre_8kHz_Test_Augmented_p1.wav", 8000, cut_and_augment_test_audio(signalTEMP,1).astype(np.int16))
wav.write("datasetTrainTestToAugment/autre_8kHz_Test_Augmented_p05.wav", 8000, cut_and_augment_test_audio(signalTEMP,0.5).astype(np.int16))